# Lab 12: Grid Localization using Bayes Filter (Virtual Robot)

### <span style="color:rgb(0,150,0)">It is recommended that you close any heavy-duty applications running on your system while working on this lab.</span>

<hr>


In [1]:
%load_ext autoreload
%autoreload 2

import traceback
from notebook_utils import *
from Traj import *
import asyncio
from localization_extras import Localization

# Setup Logger
LOG = get_logger('demo_notebook.log')

# Init GUI and Commander
gui = GET_GUI()
cmdr = gui.launcher.commander

gui.show()

# Start the simulator
START_SIM()

# Start the plotter
START_PLOTTER()

2022-04-26 14:53:51,012 | INFO     |: Logger demo_notebook.log initialized.


TwoByTwoLayout(children=(Label(value='Simulator', layout=Layout(grid_area='top-left', width='80px')), HBox(chi…

In [2]:
# Initialize Robot to communicate with the virtual robot and plotter
robot = VirtualRobot(cmdr)

# Initialize mapper
# Requires a VirtualRobot object as a parameter
mapper = Mapper(robot)

# Initialize your BaseLocalization object
# Requires a VirtualRobot object and a Mapper object as parameters
loc = Localization(robot, mapper)

## Plot Map
cmdr.plot_map()

2022-04-26 14:53:55,297 | INFO     |:  | Number of observations per grid cell: 18
2022-04-26 14:53:55,299 | INFO     |:  | Precaching Views...


/Users/main/Documents/Classes/2022-sp/ECE-4960/ece4960-fast-robots-code/lab10/ECE4960-sim-release-release-mirror/localization.py:150: RuntimeWarning: All-NaN slice encountered
  return np.nanmin(distance_intersections_tt), intersections_tt[np.nanargmin(distance_intersections_tt)]


2022-04-26 14:54:00,686 | INFO     |:  | Precaching Time: 5.386 secs
2022-04-26 14:54:00,687 | INFO     |: Initializing beliefs with a Uniform Distribution
2022-04-26 14:54:00,689 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107


# Run the Bayes Filter
The cells below utilizes the member functions of class **Localization** (defined in [localization_extras.py](../localization_extras.py)) in each iteration of the Bayes filter algorithm to localize the robot in the grid map. <br>

In [3]:
# Reset Robot and Plots
robot.reset()
cmdr.reset_plotter()

# Init Uniform Belief
loc.init_grid_beliefs()

# Get Observation Data by executing a 360 degree rotation motion
loc.get_observation_data()

# Run Update Step
loc.update_step()
loc.print_update_stats(plot_data=True)

# Plot Odom and GT
current_odom, current_gt = robot.get_pose()
cmdr.plot_gt(current_gt[0], current_gt[1])
cmdr.plot_odom(current_odom[0], current_odom[1])

2022-04-26 14:54:04,416 | INFO     |: Initializing beliefs with a Uniform Distribution
2022-04-26 14:54:04,417 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107
2022-04-26 14:54:07,325 | INFO     |: Update Step
2022-04-26 14:54:07,329 | INFO     |:      | Update Time: 0.003 secs
2022-04-26 14:54:07,330 | INFO     |: ---------- UPDATE STATS -----------
2022-04-26 14:54:07,337 | INFO     |: GT index      : (6, 4, 9)
2022-04-26 14:54:07,338 | INFO     |: Bel index     : (5, 4, 9) with prob = 0.9999999
2022-04-26 14:54:07,340 | INFO     |: Bel_bar prob at index = 0.00051440329218107
2022-04-26 14:54:07,341 | INFO     |: GT            : (0.000, 0.000, 360.000)
2022-04-26 14:54:07,343 | INFO     |: Belief        : (0.000, 0.000, 10.000)
2022-04-26 14:54:07,344 | INFO     |: POS ERROR     : (-0.000, -0.000, 350.000)
2022-04-26 14:54:07,345 | INFO     |: ---------- UPDATE STATS -----------


In [4]:
# Initialize the Trajectory object
traj = Trajectory(loc)

# Run through each motion steps
for t in range(0, traj.total_time_steps):
    print("\n\n-----------------", t, "-----------------")
    
    prev_odom, current_odom, prev_gt, current_gt = traj.execute_time_step(t)
        
    # Prediction Step
    loc.prediction_step(current_odom, prev_odom)
    loc.print_prediction_stats(plot_data=True)
    
    # Get Observation Data by executing a 360 degree rotation motion
    loc.get_observation_data()
    
    # Update Step
    loc.update_step()
    loc.print_update_stats(plot_data=True)

# Uncomment the below line to wait for keyboard input between each iteration.
#   input("Press Enter to Continue")
        
    print("-------------------------------------")



----------------- 0 -----------------
2022-04-26 14:54:16,212 | INFO     |: Prediction Step
2022-04-26 14:54:16,293 | INFO     |:  | Prediction Time: 0.080 secs
2022-04-26 14:54:16,294 | INFO     |: ---------- PREDICTION STATS -----------
2022-04-26 14:54:16,309 | INFO     |: GT index         : (6, 3, 7)
2022-04-26 14:54:16,310 | INFO     |: Prior Bel index  : (4, 4, 8) with prob = 0.1866702
2022-04-26 14:54:16,311 | INFO     |: POS ERROR        : (0.591, -0.090, -20.462)
2022-04-26 14:54:16,312 | INFO     |: ---------- PREDICTION STATS -----------
2022-04-26 14:54:19,231 | INFO     |: Update Step
2022-04-26 14:54:19,235 | INFO     |:      | Update Time: 0.003 secs
2022-04-26 14:54:19,236 | INFO     |: ---------- UPDATE STATS -----------
2022-04-26 14:54:19,243 | INFO     |: GT index      : (6, 3, 7)
2022-04-26 14:54:19,244 | INFO     |: Bel index     : (5, 3, 7) with prob = 0.9999999
2022-04-26 14:54:19,245 | INFO     |: Bel_bar prob at index = 1.2240614099036999e-20
2022-04-26 14:5